In [12]:
import numpy as np
import pandas as pd
from pypfopt import EfficientFrontier,risk_models,expected_returns,objective_functions
from pypfopt.base_optimizer import BaseConvexOptimizer
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [13]:
df_info = pd.read_excel('trading-game-data-08112023.xlsx',sheet_name='info')
df_price = pd.read_excel('trading-game-data-08112023.xlsx',sheet_name='price')
df_price.set_index('Date', inplace=True)
df_sp = pd.read_excel('trading-game-data-08112023.xlsx',sheet_name='index-price' )
df_sp.set_index('Date', inplace=True)
df_size = pd.read_excel('trading-game-data-08112023.xlsx',sheet_name='size' )
df_size.set_index('Date', inplace=True)
df_ptb = pd.read_excel('trading-game-data-08112023.xlsx',sheet_name='price-to-book' )
df_ptb.set_index('Date', inplace=True)
df_turnover = df_size = pd.read_excel('trading-game-data-08112023.xlsx',sheet_name='turnover' )
df_turnover.set_index('Date', inplace=True)

In [14]:
log_returns = np.log(df_price / df_price.shift(1))
expected_return = log_returns.mean()*252
cov_matrix = log_returns.cov()*252
rf = 0.05

In [6]:
unique_industries = df_info['RBICS Economy'].unique()
def optm_func(weights, expected_return, cov_matrix,rf):
    portfolio_return = weights @ expected_return
    portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
    sharpe_ratio = (portfolio_return - rf) / portfolio_risk
    return -sharpe_ratio

for industry in unique_industries:
    companies_in_industry = df_info[df_info['RBICS Economy'] == industry].Ticker
    df_price_industry = df_price.loc[:, companies_in_industry]
    industry_log_returns = np.log(df_price_industry / df_price_industry.shift(1))
    industry_expected_return = industry_log_returns.mean()*252
    industry_cov_matrix = industry_log_returns.cov()*252

    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
                   {'type': 'ineq', 'fun': lambda weights: weights})
   
    n_assets = len(industry_expected_return)
    bounds = [(0, 1) for _ in range(n_assets)]

    initial_weights = np.ones(n_assets) / n_assets

    optimization = minimize(optm_func, initial_weights, args=(industry_expected_return, industry_cov_matrix, rf),
                    constraints=constraints, bounds=bounds)
    optimal_weights = optimization.x
    optimal_return = optimal_weights @ industry_expected_return
    optimal_risk = np.sqrt(optimal_weights @ industry_cov_matrix @ optimal_weights)
    optimal_sharpe_ratio = (optimal_return - rf) / optimal_risk
    optimal_weights = np.round(optimal_weights,4)

    #print(f"{industry} Optimal Portfolio Weights:", optimal_weights)
    print(f"{industry} Portfolio Return:", optimal_return)
    print(f"{industry} Portfolio Risk:", optimal_risk)
    #print(f"{industry} Sharpe Ratio:", optimal_sharpe_ratio)

Healthcare Portfolio Return: 0.43494036646553813
Healthcare Portfolio Risk: 0.16389615536265262
Industrials Portfolio Return: 0.5285898396220424
Industrials Portfolio Risk: 0.19063610055853383
Technology Portfolio Return: 0.9796303841639803
Technology Portfolio Risk: 0.2870331619150057
Consumer Services Portfolio Return: 0.5633876003495777
Consumer Services Portfolio Risk: 0.21676737955665276
Finance Portfolio Return: 0.3358156554024213
Finance Portfolio Risk: 0.12421037873069753
Consumer Non-Cyclicals Portfolio Return: 0.3743506728699406
Consumer Non-Cyclicals Portfolio Risk: 0.18030546934970948
Business Services Portfolio Return: 0.3842668688355223
Business Services Portfolio Risk: 0.1598384499358929
Utilities Portfolio Return: 0.40398912081023153
Utilities Portfolio Risk: 0.2237404799144419
Non-Energy Materials Portfolio Return: 0.26668959850957424
Non-Energy Materials Portfolio Risk: 0.19169678045842442
Energy Portfolio Return: 0.2576859941249493
Energy Portfolio Risk: 0.2865805267

In [7]:
def optm_func(weights, expected_return, cov_matrix,rf):
    portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
    portfolio_return = weights @ expected_return
    adjusted_sharpe_ratio = (portfolio_return - rf) / portfolio_risk
    return -adjusted_sharpe_ratio

constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
               {'type': 'ineq', 'fun': lambda weights: weights})

n_assets = len(expected_return)
bounds = [(0, 1) for _ in range(n_assets)]
initial_weights = np.ones(n_assets) / n_assets

optimization = minimize(optm_func, initial_weights, args=(expected_return, cov_matrix, rf),
                constraints=constraints, bounds=bounds)
optimal_weights = optimization.x
optimal_return = optimal_weights @ expected_return
optimal_risk = np.sqrt(optimal_weights @ cov_matrix @ optimal_weights)
optimal_sharpe_ratio = (optimal_return - rf) / optimal_risk
optimal_weights = np.round(optimal_weights,4)

print(optimal_weights)
print("Optimal Portfolio Return:", optimal_return)
print("Optimal Portfolio Risk:", optimal_risk)


[0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.2478 0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.0061 0.0381 0.     0.     0.     0.     0.     0.
 0.     0.0772 0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.   

In [8]:
# target_returns = np.arange(0.1,0.7,0.05)
# portfolio_returns = []
# portfolio_risks = []
# for target_return in target_returns:
#     constraint_return = {'type': 'eq', 'fun': lambda weights: weights @ expected_return - target_return}
#     constraints = [constraint_return] + [{'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
#                {'type': 'ineq', 'fun': lambda weights: weights}]
#     optimization = minimize(optm_func, initial_weights, args=(expected_return, cov_matrix, rf),
#                             constraints=constraints, bounds=bounds)
#     optimal_weights = optimization.x
#     portfolio_risk = np.sqrt(optimal_weights @ cov_matrix @ optimal_weights)

#     portfolio_returns.append(target_return)
#     portfolio_risks.append(portfolio_risk)
#     print(target_return)
    
# portfolio_returns = np.array(portfolio_returns)
# portfolio_risks = np.array(portfolio_risks)


In [9]:
# plt.plot(portfolio_risks[2:],portfolio_returns[2:]-0.25*portfolio_risks[2:])
# plt.plot(portfolio_risks[2:],portfolio_returns[2:],color='red')

In [16]:
df_price_tails = df_price
mu = expected_returns.mean_historical_return(df_price_tails, log_returns=True,frequency=252,compounding=False)
S = risk_models.sample_cov(df_price_tails, log_returns=True,frequency=252)
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe(risk_free_rate=0.05)
cleaned_weights = list(ef.clean_weights().values())
print(cleaned_weights)
ef.portfolio_performance(verbose=True,risk_free_rate=0.05)
ef.save_weights_to_file('weights.csv')

# def optm(weights, expected_returns, cov_matrix):
#     portfolio_return = weights @ mu
#     portfolio_risk = np.dot(weights, np.dot(cov_matrix, weights))
#     return 0.8*portfolio_return - 0.25 * portfolio_risk

# w = ef.convex_objective(optm,expected_returns = mu,cov_matrix = S)

C:\Users\Jiaqi\AppData\Roaming\Python\Python311\site-packages\pypfopt\risk_models.py:70: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(
C:\Users\Jiaqi\AppData\Roaming\Python\Python311\site-packages\pypfopt\risk_models.py:89: UserWarning: Could not fix matrix. Please try a different risk model.
  warnings.warn(


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.24831, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00596, 0.03766, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.07598, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

C:\Users\Jiaqi\AppData\Roaming\Python\Python311\site-packages\cvxpy\problems\problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [ ]:
S